In [ ]:

from requests import post
from pydantic import BaseModel
from trilogy import Dialects


In [ ]:
class ModelSourceInSchema(BaseModel):
    alias: str
    contents: str


class ModelInSchema(BaseModel):
    name: str
    sources: list[ModelSourceInSchema]
    

class QueryInSchema(BaseModel):
    query: str
    dialect = Dialects
    full_model: ModelInSchema | None = None

In [21]:
import datetime

request = QueryInSchema(
    query = '''
select 1-> test;''',
    dialect='bigquery'
)
start = datetime.datetime.now()
print(request.model_dump_json())
x = post('http://localhost:5678/generate_query', data=request.model_dump_json()).json()
print(datetime.datetime.now() - start)
print(x)
print(x['generated_sql'])


{"query":"\nselect 1-> test;","full_model":null}
0:00:02.055650
{'detail': [{'type': 'missing', 'loc': ['body', 'dialect'], 'msg': 'Field required', 'input': {'query': '\nselect 1-> test;', 'full_model': None}}]}


KeyError: 'generated_sql'